# rating-gp (prototype)
`rating-gp` is a prototype model that can fit rating curves (stage-discharge relationship) using the Generalized Power Law Model (GPLM) of [Hrafnkelsson et al. (2021)](https://doi.org/10.1002/env.2711) using a Gaussian process.
This model seeks to expand the GPLM to include changes in the rating curve with time such that the time evolution in the rating curve can be included in the model.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thodson-usgs/discontinuum/blob/main/notebooks/rating-gp-demo.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/thodson-usgs/discontinuum/main?labpath=notebooks%2Frating-gp-demo.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline

In [ ]:
# SF Coeur D Alene River nr Pinehurst, ID
site = "12413470"
start_date = "1988-10-01" 
end_date = "2021-09-30"

First, download the data. In `discontinuum`, the convention is to download directly using `providers`, which wrap a data provider's web-service and perform some initial formatting and metadata construction, then return the result as an `xarray.Dataset`. Here, we'll uses the `usgs` provider. If you need data from another source, create a `provider` and ensure the output matches that of the `usgs` provider. Here, we'll download some instantaneous stage data to use as our model's input, and some discharge data as our target. 

In [ ]:
from rating_gp.providers import usgs

# download instantaneous stage and discharge measurements
training_data = usgs.get_measurements(site=site, start_date=start_date, end_date=end_date)
training_data

In [ ]:
training_data.plot.scatter(x="stage", y="discharge")

With the training data, we're now ready to fit the model. Depending on your hardware, this should only take about 10-20s. The first fit will also compile the model, which takes longer. After running it once, try running the cell again and note the difference in wall time.

In [ ]:
%%time
# select an engine
from rating_gp.models.gpytorch import RatingGPMarginalGPyTorch as RatingGP

model = RatingGP()
model.fit(target=training_data['discharge'], covariates=training_data[['stage']], target_unc=training_data['discharge_unc'])

With the model fit, we can generate some nice plots of the rating curve and time series of both stage and discharge.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 7), sharex='col', sharey='row')
ax[0, 0] = model.plot_stage(ax=ax[0, 0])
ax[1, 0] = model.plot_observations(ax=ax[1, 0])
ax[1, 1] = model.plot_observed_rating(ax=ax[1, 1])
ax[0, 1].axis('off')

Now, let's download some dialy stage measurements and see how the predictions for discharge look for these measurements.

In [ ]:
test_data = usgs.get_daily_stage(site=site, start_date=start_date, end_date=end_date)
test_data

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 7), sharex='col', sharey='row')
ax[0, 0] = model.plot_stage(test_data, ax=ax[0, 0])
ax[1, 0] = model.plot(test_data, ax=ax[1, 0])
ax[1, 1] = model.plot_rating(test_data.sortby("stage"), ax=ax[1, 1])
ax[0, 1].axis('off')

These results look pretty good in the time series plots. The fluctuations in the rating curve are expected as we are modeling the changes in the rating curve with time. Therefore, the fluctuations are the due to changes in the rating curve.